In [ ]:
# Import LangGraph and LangChain components
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain.agents import create_agent

# Imports that enable human-in-the-loop implementation
from langgraph.types import Command, interrupt

In [ ]:
import os
import logging

from bedrock_agentcore.memory import MemoryClient
# Import the AgentCoreMemorySaver that we will use as a checkpointer
from langgraph_checkpoint_aws import AgentCoreMemorySaver

from dotenv import load_dotenv

load_dotenv()

logging.getLogger("support-agent").setLevel(logging.INFO)
region = os.getenv('AWS_REGION', 'us-west-2')

logger = logging.getLogger("support-agent")

In [ ]:
memory_name = "SupportAgent"

client = MemoryClient(region_name=region)
memory = client.create_or_get_memory(name=memory_name)
memory_id = memory["id"]

In [ ]:
MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"

# Initialize checkpointer for state persistence
checkpointer = AgentCoreMemorySaver(memory_id, region_name=region)

In [ ]:
@tool
def human_assistance(query: str) -> str:
    """Request assistance from a human."""
    human_response = interrupt({"query": query})
    return human_response["data"]


@tool
def add(a: int, b: int):
    """Add two integers and return the result"""
    return a + b


@tool
def multiply(a: int, b: int):
    """Multiply two integers and return the result"""
    return a * b


tools = [add, multiply, human_assistance]

In [ ]:
# Initialize LLM
llm = init_chat_model(MODEL_ID, model_provider="bedrock_converse", region_name=region)

graph = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful assistant",
    checkpointer=checkpointer,
)

graph

In [ ]:
user_input = "I would like to work with a customer service human agent."
config = {"configurable": {"thread_id": "1", "actor_id": "demo-notebook"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
snapshot = graph.get_state(config)
snapshot.next

In [ ]:
human_response = (
    "I'm sorry to hear that you are frustrated. Looking at the past conversation history, I can see that you've requested a refund. I've gone ahead and credited it to your account."
)

human_command = Command(resume={"messages": human_response})

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()